# Project Introduction
listen to samples of the MUSDB18 dataset

In [1]:
import musdb
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio, display, clear_output
import ipywidgets as widgets
from pathlib import Path

# =============================================================================
# 1. SETUP & DATA LOADING
# =============================================================================

print("Loading MUSDB18 Dataset...")

# This will look for the full dataset. 
# If not found, it DOWNLOADS the 7-second demo snippets (~50MB) automatically.
try:
    mus = musdb.DB(download=True)
    tracks = mus.tracks
    print(f" Successfully loaded {len(tracks)} tracks.")
except Exception as e:
    print(f" Error loading MUSDB18: {e}")
    print("Please ensure 'ffmpeg' is installed (conda install -c conda-forge ffmpeg)")
    tracks = []

# =============================================================================
# 2. INTERACTIVE PLAYER LOGIC
# =============================================================================

# -- Widgets --
style = {'description_width': 'initial'}

# Dropdown for Song Selection
track_options = [(f"{i}: {t.title} - {t.artist}", i) for i, t in enumerate(tracks)]
song_dropdown = widgets.Dropdown(
    options=track_options,
    description='Song:',
    style=style,
    layout=widgets.Layout(width='50%')
)

# Dropdown for Stem Selection
stem_dropdown = widgets.Dropdown(
    options=['mixture', 'vocals', 'drums', 'bass', 'other'],
    value='mixture',
    description='Stem:',
    style=style,
    layout=widgets.Layout(width='30%')
)

# Output Area for Player & Plots
out_display = widgets.Output()

def load_and_play(change=None):
    """Callback to load audio and display widgets"""
    if not tracks: return

    out_display.clear_output(wait=True)
    
    # Get current selections
    track_idx = song_dropdown.value
    stem_name = stem_dropdown.value
    
    track = tracks[track_idx]
    
    # Load Audio
    # Note: musdb tracks are objects; we access .audio or .targets[stem].audio
    if stem_name == 'mixture':
        audio = track.audio  # Shape: (samples, 2)
    else:
        audio = track.targets[stem_name].audio
    
    # Convert Stereo to Mono for simple visualization
    audio_mono = np.mean(audio, axis=1)
    sr = track.rate

    with out_display:
        print(f" Now Playing: {track.title} [{stem_name.upper()}]")
        
        # 1. Audio Player
        # We take just the first 10 seconds to keep the notebook light if using full tracks
        # (The snippets are only 7s anyway)
        display(Audio(data=audio_mono, rate=sr))
        
        # 2. Visualization (Waveform + Spectrogram)
        fig, axes = plt.subplots(1, 2, figsize=(14, 4))
        
        # Waveform
        librosa.display.waveshow(audio_mono, sr=sr, ax=axes[0], alpha=0.6)
        axes[0].set_title(f"Waveform ({stem_name})")
        axes[0].set_xlabel("Time (s)")
        
        # Spectrogram
        S = librosa.stft(audio_mono)
        S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
        img = librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log', ax=axes[1], cmap='magma')
        fig.colorbar(img, ax=axes[1], format="%+2.0f dB")
        axes[1].set_title(f"Spectrogram ({stem_name})")
        
        plt.tight_layout()
        plt.show()

# -- Link Widgets to Callback --
song_dropdown.observe(load_and_play, names='value')
stem_dropdown.observe(load_and_play, names='value')

# -- Display Interface --
ui = widgets.VBox([
    widgets.HBox([song_dropdown, stem_dropdown]),
    out_display
])

display(ui)

# Trigger first load
if tracks:
    load_and_play()

Loading MUSDB18 Dataset...
 Successfully loaded 144 tracks.
